In [9]:
import pandas as pd
import progressbar
from collections import defaultdict
df=pd.read_csv('z_250.csv',delim_whitespace=True)


In [10]:
def enumerate_types(df):
    enum_dict = defaultdict(int) #{}
    enum_counter = 0
    for i, type in enumerate(df['type']):
        if type not in enum_dict:
            enum_dict[type] = enum_counter
            enum_counter += 1
        df.at[i, 'type_id'] = enum_dict[type]
    df['type_id'] = df['type_id'].astype(int)
    return df
df=enumerate_types(df)

In [11]:
df

,#doc,source,pos,typeindex,type,topic,type_id
0,0,NaN,0,0,"Induction,",47,0
1,0,NaN,1,1,detection,29,1
2,0,NaN,2,2,isolation,39,2
3,0,NaN,3,3,temperature-sensitive,39,3
4,0,NaN,4,4,lethal,39,4
...,...,...,...,...,...,...,...
653248,6214,NaN,119,3969,turnover,11,3959
653249,6214,NaN,120,8305,binding,26,8226
653250,6214,NaN,121,5195,unique,36,5116
653251,6214,NaN,122,28578,interface,11,28454


In [12]:
#dictionary
def get_dictionary(df):
    dictionary=df[['type_id','type']].drop_duplicates(subset='type_id').reset_index(drop=True)
    return dictionary

In [13]:
get_dictionary(df)

,type_id,type
0,0,"Induction,"
1,1,detection
2,2,isolation
3,3,temperature-sensitive
4,4,lethal
...,...,...
68051,68051,CLIK(23)
68052,68052,melanoma
68053,68053,adenocarcinoma
68054,68054,metastasising


In [14]:
# Document-topic matrix
def get_document_topic(df):
    #Groupby topics for each document, creating new column denoting the frequency
    doc_topic = df.groupby(['#doc', 'topic']).size().reset_index(name='frequency')

    # Calculate total frequencies for each document
    total_frequencies = doc_topic.groupby('#doc')['frequency'].sum().reset_index()

    # join total frequencies into the doc_topic dataframe
    doc_topic = doc_topic.merge(total_frequencies, on='#doc')

    # Calculate the weight
    doc_topic['weight'] = doc_topic['frequency_x'] / doc_topic['frequency_y']

    doc_topic = doc_topic.drop(['frequency_x','frequency_y'], axis=1)

    #Drop topics which have very low weight
    doc_topic.drop(doc_topic[doc_topic['weight']<0.001].index,inplace=True)
    
    return doc_topic

In [15]:
get_document_topic(df)

,#doc,topic,weight
0,0,5,0.021739
1,0,9,0.021739
2,0,12,0.043478
3,0,15,0.043478
4,0,21,0.065217
...,...,...,...
70550,6214,32,0.048387
70551,6214,36,0.008065
70552,6214,38,0.096774
70553,6214,45,0.008065


In [16]:
# Topic-type matrix
def get_topic_type(df):
    #Groupby types for each topic, creating new column denoting the frequency
    topic_type = df.groupby(['topic', 'type_id']).size().reset_index(name='frequency')

    # Calculate total frequencies for each topic
    total_frequencies = topic_type.groupby('topic')['frequency'].sum().reset_index()

    # join total frequencies into the topic_type dataframe
    topic_type = topic_type.merge(total_frequencies, on='topic')

    # Calculate the weight
    topic_type['weight'] = topic_type['frequency_x'] / topic_type['frequency_y']

    #drop frquency columns
    topic_type = topic_type.drop(['frequency_x','frequency_y'], axis=1)

    #Drop types which have very low weight
    topic_type.drop(topic_type[topic_type['weight']<0.001].index,inplace=True)

    return topic_type

In [17]:
get_topic_type(df)

,topic,type_id,weight
1,0,10,0.019600
2,0,12,0.004337
3,0,13,0.011437
4,0,26,0.009481
5,0,27,0.005825
...,...,...,...
95237,49,48465,0.001092
95247,49,48905,0.001248
95328,49,52012,0.001014
95350,49,52307,0.001014
